In [1]:
from music21 import converter, instrument, note, chord, stream
import glob
import pickle
import numpy as np
from tensorflow.keras.utils import to_categorical

## Extracting notes and chords from MIDI files

In [116]:
files = glob.glob('./maestro-v2.0.0/2013/*')

In [118]:
notes = []
for i, file in enumerate(files):
    print('reading file:',i, ';len_notes:', len(notes), end = '\r')
    if len(notes) > 30000:
        break
    midi = converter.parse(file)
    
    elements_to_parse = midi.flat.notes
    
    for ele in elements_to_parse:
        
        if isinstance(ele, note.Note):
            notes.append(str(ele.pitch))
        
        elif isinstance(ele, chord.Chord):
            notes.append('+'.join(str(n) for n in ele.normalOrder))
        
        else:
            print('found something else')
            print(ele)
            print('this was it')

In [121]:
# len(set(notes)), len(notes)

In [122]:
# import pickle
# with open('notes-2', 'wb') as file:
#     pickle.dump(notes, file)

In [157]:
with open('notes', 'rb') as file:
    notes = pickle.load(file)

## Preparing seauence data

In [158]:
seq_len = 100

In [159]:
pitchnames = sorted(set(notes))
n_vocab = len(pitchnames)
len(notes), len(set(notes))

(62906, 600)

In [160]:
# Dictionary to map notes with integer
ele_to_int = dict([(ele, i) for i, ele in enumerate(pitchnames)])

In [161]:
network_input = []
network_output = []

In [162]:
# Creating Sequence Data
for i in range(len(notes) - seq_len):
    seq_in = notes[i:i+seq_len]
    seq_out = notes[i+seq_len]
    
    network_input.append([ele_to_int[ele] for ele in seq_in])
    network_output.append(ele_to_int[seq_out])

In [163]:
n_patterns = len(network_input)
print(n_patterns)

62806


In [164]:
network_input = np.reshape(network_input, (n_patterns, seq_len, 1))
network_input.shape

(62806, 100, 1)

In [165]:
# Normalising Input Data...
normalised_network_input = network_input/float(n_vocab)

In [166]:
# Getting One-Hot vector Representation for supervised learning...
network_output = to_categorical(network_output)
network_output.shape

(62806, 600)

In [167]:
print(normalised_network_input.shape)
print(network_output.shape)

(62806, 100, 1)
(62806, 600)


 ## Create Model

In [168]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [169]:
model = Sequential()
model.add(LSTM(units = 512, 
        input_shape = (normalised_network_input.shape[1], normalised_network_input.shape[2]),
        return_sequences = True))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences = True))
model.add(Dropout(0.3))
model.add(LSTM(512))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab, activation = 'softmax'))

model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')

In [170]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 100, 512)          1052672   
_________________________________________________________________
dropout_3 (Dropout)          (None, 100, 512)          0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 100, 512)          2099200   
_________________________________________________________________
dropout_4 (Dropout)          (None, 100, 512)          0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_5 (Dropout)          (None, 256)              

In [171]:
# checkpoint = ModelCheckpoint('./models/model.hdf5', monitor = 'loss', verbose = 1,
#                              save_best_only=True, mode = 'min')

# hist_fit = model.fit(normalised_network_input, network_output, batch_size = 64, 
#                      epochs = 100, callbacks = [checkpoint])

## Predicting

In [173]:
# Load model
model = load_model('./models/model-5.hdf5')
seq_len = 100

In [174]:
# Prepare sample input for prediction and generate model predictions...
def get_model_ouput(model, notes, seq_len, start = None, num_notes = 100):
    
    pitchnames = sorted(set(notes))
    n_vocab = len(pitchnames)
    len(notes), len(set(notes))
    
    network_input = []
    for i in range(len(notes) - seq_len):
        seq_in = notes[i:i+seq_len]
        network_input.append([ele_to_int[ele] for ele in seq_in])
   
    if start is None:
        start = np.random.randint(len(network_input) - 1)
    
#     Used Later for debugging and validation.
    print('start:', start)
    
    int_to_ele = dict([(i, ele) for i,ele in enumerate(pitchnames)])
    pattern = network_input[start]

    prediction_output = []
    
    for i in range(num_notes):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input/n_vocab

        prediction = model.predict(prediction_input)
        idx = np.argmax(prediction)

        result = int_to_ele[idx]

        prediction_output.append(result)

        pattern.append(idx)
        pattern = pattern[1:]
        if i%10 == 0:
            print(i, end = '\r')
        
    return prediction_output

In [185]:
prediction_output = get_model_ouput(model, notes, 100, num_notes=100)

start: 1252


In [186]:
len(prediction_output)

100

In [187]:
# prediction_output

## Create MIDI File

In [188]:
from music21.instrument import Instrument

In [189]:
def get_output_notes(prediction_output):
    offset = 0
    output_notes = []
    k = 0
    last = len(prediction_output)
    for pattern in prediction_output:

        # if pattern is chord
        if ('+' in pattern) or (pattern.isdigit()):
            notes_in_chord = pattern.split('+')
            temp_notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                temp_notes.append(new_note)
            new_chord = chord.Chord(temp_notes) 
            new_chord.offset = offset
            output_notes.append(new_chord)
            continue

        #if pattern is note
        new_note = note.Note(pattern)
        new_note.storedInstrument = instrument.Piano()
        new_note.offset = offset
        output_notes.append(new_note)

        if k < 5:
            offset += 1.0
            print(k)
        else:
            offset += 0.5
        k += 1
    
    k = 5
    while k > 0:
        if isinstance(output_notes[-k], note.Note):
            output_notes[-k].offset = output_notes[-k-1].offset + 1.0
        k -= 1
        print(k)
            
    return output_notes

In [190]:
output_notes = get_output_notes(prediction_output)
len(output_notes)

0
1
2
3
4
4
3
2
1
0


100

In [191]:
def get_midi_file(output_notes, save = False, file_name = None):
    
    midi_stream = stream.Stream(output_notes)
    
    if save == True:
        midi_stream.write('midi', fp = file_name)
    
    return midi_stream

In [192]:
midi_stream = get_midi_file(output_notes)
midi_stream.show('midi')

## Checking for overfitting

In [193]:
def check(prediction_output, network_input):
    inp = list(network_input[start].copy().reshape(-1, ))
    ls = list(network_input[0].copy().reshape(-1, ))

    for out in network_output:
        ls.append(np.argmax(out.copy()))
    
    for i in range(len(ls)):
        if ls[i:100 + i] == inp:
            print('found')
            break
    
    print('Now checking for overfitting')
    
    n = len(prediction_output)
    count = 0
    
    preds = []
    for i in prediction_output:
        preds.append(ele_to_int[i])
    
    for i, ele in enumerate(ls[start + 100: start + 100 + n]):
        if ele != preds[i]:
            count += 1
    print(f'{n - count} out of {n} are same.')

In [194]:
check(prediction_output, network_input)

found
Now checking for overfitting
3 out of 100 are same.
